# Encode data

This notebook runs part of the Multi-Omics Variational autoEncoder (MOVE) framework for using the structure the VAE has identified for extracting categorical data assositions across all continuous datasets. In the MOVE paper we used it for identifiying drug assosiations in clinical and multi-omics data. This part is a guide for encoding the data that can be used as input in MOVE. 

In [1]:
# Import functions
from hydra import initialize, compose

from move.utils.data_utils import read_ids, generate_file, merge_configs 

For encoding the data you need to have each dataset/data type in a format for N x M, where N is the numer of samples/individuals and M is the number of features. For using the dataset specific weighting in the training of the VAE you need to process the datasets individually or split them when you read them in. The continuous data is z-score normalised and the categorical data is one-hot encoded. Below is an example of processing a continuous dataset and two categorical datasets with different number of categories. To ensure the correct order the ID's are used for sorting the data accordingly.

In [2]:
# Initializing the default config 
with initialize(version_base=None, config_path="src/move/conf"):
    base_config = compose(config_name="main")

def main(base_config=base_config):
    
    # Overriding base_config with the user defined configs.
    cfg = merge_configs(base_config=base_config, 
                        config_types=['data'])
    
    # Getting the variables used in the notebook
    raw_data_path = cfg.data.raw_data_path
    interim_data_path = cfg.data.interim_data_path
    ids_file_name = cfg.data.ids_file_name
    ids_has_header = cfg.data.ids_has_header
    ids_colname = cfg.data.ids_colname
    
    na_encoding = cfg.data.na_value
    categorical_names = cfg.data.categorical_names
    continuous_names = cfg.data.continuous_names    
    
    # Reading ids 
    ids = read_ids(raw_data_path, ids_file_name, ids_colname, ids_has_header)

    # Encoding categorical data
    print('Encoding categorical data')
    for cat_data in categorical_names:
        generate_file('categorical', raw_data_path, interim_data_path, cat_data, ids, na_encoding)
        print(f'  Encoded {cat_data}')
    
    # Encoding continuous data 
    print('Encoding continuous data')
    for con_data in continuous_names:
        generate_file('continuous', raw_data_path, interim_data_path, con_data, ids, na_encoding)    
        print(f'  Encoded {con_data}')

if __name__ == "__main__":
    main()

Overriding the default config with configs from data.yaml

Configuration used: 
---
data:
  user_config: data.yaml
  na_value: na
  raw_data_path: data/
  interim_data_path: interim_data/
  processed_data_path: processed_data/
  version: v1
  ids_file_name: baseline_ids
  ids_has_header: false
  ids_colname: 0
  categorical_inputs:
  - name: diabetes_genotypes
    weight: 1
  - name: baseline_drugs
    weight: 1
  - name: baseline_categorical
    weight: 1
  continuous_inputs:
  - name: baseline_continuous
    weight: 2
  - name: baseline_transcriptomics
    weight: 1
  - name: baseline_diet_wearables
    weight: 1
  - name: baseline_proteomic_antibodies
    weight: 1
  - name: baseline_target_metabolomics
    weight: 1
  - name: baseline_untarget_metabolomics
    weight: 1
  - name: baseline_metagenomics
    weight: 1
  data_of_interest: baseline_drugs
  categorical_names: ${names:${data.categorical_inputs}}
  continuous_names: ${names:${data.continuous_inputs}}
  categorical_weights:

In [3]:
# import pandas as pd
# raw_data_path='data/'
# ids_file_name='baseline_ids'
# ids_has_header=True
# ids_colname='0'

# def read_ids(path, ids_file_name, ids_colname, ids_has_header=True):
#     header=0 if ids_has_header else None
        
#     ids = pd.read_csv(path + f"{ids_file_name}.txt", sep='\t', header=header)
#     ids.columns = ids.columns.astype(str)
    
#     return list(ids[ids_colname])

# ids = read_ids(raw_data_path, ids_file_name, ids_colname, ids_has_header)
# print(ids)